In [2]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
import tempfile # For robust temporary file handling

# --- Configuration ---
# Define the output directory and database name at the top for easy access.
XYZ_DIR = Path("pubchem_xyz_structures")
DB_FILE = Path("molecules_pubchem.db")

# Just add any molecule names you want to this list!
MOLECULE_NAMES = [
    "methane", "water", "ammonia", "carbon dioxide", "ethane",
    "benzene", "acetone", "ethanol", "acetic acid", "glucose", 
    "hexane", "caffeine", "aspirin" # Added more examples
]

# --- Functions ---

def download_and_convert_to_xyz(molecule_name, mol_id, temp_dir):
    """Downloads a molecule from PubChem, converts it to XYZ format, and saves it."""
    try:
        # Search for the molecule by name
        compounds = pcp.get_compounds(molecule_name, 'name')
        if not compounds:
            print(f"❌ Molecule '{molecule_name}' not found on PubChem.")
            return None
        
        compound = compounds[0]
        
        # Download the 3D structure in SDF format to a temporary, cross-platform location
        temp_sdf_path = Path(temp_dir) / f"{compound.cid}.sdf"
        pcp.download('SDF', str(temp_sdf_path), compound.cid, record_type='3d', overwrite=True)
        
        # Read the SDF content
        sdf_content = temp_sdf_path.read_text()
        if not sdf_content:
            print(f"⚠️ No 3D SDF content found for {molecule_name} (CID: {compound.cid}).")
            return None

        # Load the molecule using RDKit
        mol = Chem.MolFromMolBlock(sdf_content, removeHs=False)
        if mol is None:
            print(f"❌ RDKit could not parse the SDF for {molecule_name}.")
            return None
        
        # Prepare the XYZ content
        conf = mol.GetConformer()
        xyz_lines = [str(mol.GetNumAtoms())]
        xyz_lines.append(f"{molecule_name} - PubChem CID: {compound.cid}")
        
        for i, atom in enumerate(mol.GetAtoms()):
            pos = conf.GetAtomPosition(i)
            symbol = atom.GetSymbol()
            xyz_lines.append(f"{symbol:2s} {pos.x:12.6f} {pos.y:12.6f} {pos.z:12.6f}")
        
        # Ensure the output directory exists
        XYZ_DIR.mkdir(exist_ok=True)
        
        # Save the final XYZ file
        filename = f"molecule_{mol_id}_{molecule_name.replace(' ', '_')}.xyz"
        filepath = XYZ_DIR / filename
        filepath.write_text('\n'.join(xyz_lines))
        
        print(f"✅ Downloaded and saved: {filename}")
        return str(filepath), compound.cid
        
    except Exception as e:
        print(f"❌ Error processing '{molecule_name}': {e}")
        return None

def create_database():
    """Creates the SQLite database and the molecules table if they don't exist."""
    with sqlite3.connect(DB_FILE) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS molecules (
                molecule_id INTEGER PRIMARY KEY,
                molecule_name TEXT NOT NULL UNIQUE,
                pubchem_cid INTEGER,
                free_energy REAL,
                xyz_filepath TEXT,
                molecular_formula TEXT,
                molecular_weight REAL
            )
        ''')
        conn.commit()

def populate_database(molecule_names):
    """
    Populates the database by downloading data for each molecule in the list.
    It generates random free energy data based on the list size.
    """
    print(f"\n--- Populating Database with {len(molecule_names)} Molecules ---")
    create_database()
    
    # --- DYNAMIC DATA GENERATION ---
    # The number of molecules is determined by the length of your input list.
    num_molecules = len(molecule_names)
    np.random.seed(42) # for reproducible random data
    molecule_ids = np.arange(1, num_molecules + 1)
    free_energies = -np.abs(np.random.normal(loc=50, scale=10, size=num_molecules))

    # Use a temporary directory that cleans itself up automatically
    with tempfile.TemporaryDirectory() as temp_dir, sqlite3.connect(DB_FILE) as conn:
        cursor = conn.cursor()
        
        for i, molecule_name in enumerate(molecule_names):
            mol_id = molecule_ids[i]
            free_energy = free_energies[i]
            
            print(f"\nProcessing molecule {i+1}/{num_molecules}: {molecule_name}")
            
            result = download_and_convert_to_xyz(molecule_name, mol_id, temp_dir)
            
            if result:
                filepath, cid = result
                try:
                    # Get additional metadata from PubChem
                    compound = pcp.Compound.from_cid(cid)
                    formula = compound.molecular_formula
                    weight = float(compound.molecular_weight) if compound.molecular_weight else 0.0
                    
                    # Prepare data for insertion (using standard Python types)
                    insert_data = (
                        int(mol_id), 
                        molecule_name, 
                        int(cid), 
                        float(free_energy), 
                        filepath, 
                        formula, 
                        weight
                    )

                    # INSERT OR IGNORE will skip molecules already in the DB by name
                    cursor.execute('''
                        INSERT OR REPLACE INTO molecules 
                        (molecule_id, molecule_name, pubchem_cid, free_energy, xyz_filepath, 
                         molecular_formula, molecular_weight)
                        VALUES (?, ?, ?, ?, ?, ?, ?)
                    ''', insert_data)
                    
                    print(f"-> Added '{molecule_name}' to the database.")
                    
                except Exception as e:
                    print(f"-> ❌ Database error for '{molecule_name}': {e}")
            else:
                print(f"-> Skipped '{molecule_name}' due to a download/processing error.")
        
        conn.commit() # Commit all changes at the end
    print("\n--- Database population complete! ---")

def query_database():
    """Queries and returns the entire molecules table as a pandas DataFrame."""
    if not DB_FILE.exists():
        print("Database file not found.")
        return pd.DataFrame()
        
    with sqlite3.connect(DB_FILE) as conn:
        query_df = pd.read_sql_query("SELECT * FROM molecules ORDER BY molecule_id", conn)
        return query_df

# --- Main Execution Block ---
if __name__ == "__main__":
    print("🚀 Starting PubChem downloader script...")
    
    # This will download and populate the database for all molecules in the list.
    populate_database(MOLECULE_NAMES)

    print("\nFinal database contents:")
    final_df = query_database()
    
    # Configure pandas for better display
    pd.set_option('display.max_rows', 50)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.width', 120)
    
    print(final_df)
    
    print("\n✅ Script completed successfully!")

🚀 Starting PubChem downloader script...

--- Populating Database with 13 Molecules ---

Processing molecule 1/13: methane
✅ Downloaded and saved: molecule_1_methane.xyz
-> Added 'methane' to the database.

Processing molecule 2/13: water
✅ Downloaded and saved: molecule_2_water.xyz
-> Added 'water' to the database.

Processing molecule 3/13: ammonia
✅ Downloaded and saved: molecule_3_ammonia.xyz
-> Added 'ammonia' to the database.

Processing molecule 4/13: carbon dioxide


[18:25:46] Warning: molecule is tagged as 3D, but all Z coords are zero


✅ Downloaded and saved: molecule_4_carbon_dioxide.xyz
-> Added 'carbon dioxide' to the database.

Processing molecule 5/13: ethane
✅ Downloaded and saved: molecule_5_ethane.xyz
-> Added 'ethane' to the database.

Processing molecule 6/13: benzene
✅ Downloaded and saved: molecule_6_benzene.xyz
-> Added 'benzene' to the database.

Processing molecule 7/13: acetone
✅ Downloaded and saved: molecule_7_acetone.xyz
-> Added 'acetone' to the database.

Processing molecule 8/13: ethanol
✅ Downloaded and saved: molecule_8_ethanol.xyz
-> Added 'ethanol' to the database.

Processing molecule 9/13: acetic acid
✅ Downloaded and saved: molecule_9_acetic_acid.xyz
-> Added 'acetic acid' to the database.

Processing molecule 10/13: glucose
✅ Downloaded and saved: molecule_10_glucose.xyz
-> Added 'glucose' to the database.

Processing molecule 11/13: hexane
✅ Downloaded and saved: molecule_11_hexane.xyz
-> Added 'hexane' to the database.

Processing molecule 12/13: caffeine
✅ Downloaded and saved: molecu